<a href="https://colab.research.google.com/github/path-0f-misantrope/most_uselles_musor_final/blob/main/modellll_reworked.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
!pip install scikit-learn


In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset


In [8]:
df = pd.read_csv('/content/разметка_готовая.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1970 entries, 0 to 1969
Data columns (total 6 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   comment                               1970 non-null   object 
 1   Нравится скорость отработки заявок    1970 non-null   int64  
 2   Нравится качество выполнения заявки   1969 non-null   float64
 3   Нравится качество работы сотрудников  1970 non-null   int64  
 4   Понравилось выполнение заявки         1969 non-null   float64
 5   Вопрос решен                          1967 non-null   float64
dtypes: float64(3), int64(2), object(1)
memory usage: 92.5+ KB


In [ ]:
df.info()

In [9]:

text_col = 'comment'
label_cols = [
    'Нравится скорость отработки заявок',
    'Нравится качество выполнения заявки',
    'Нравится качество работы сотрудников',
    'Вопрос решен',
    "Понравилось выполнение заявки"
]
df = df[[text_col] + label_cols].dropna()
df = df[df[label_cols].sum(axis=1) > 0]

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1671 entries, 0 to 1969
Data columns (total 6 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   comment                               1671 non-null   object 
 1   Нравится скорость отработки заявок    1671 non-null   int64  
 2   Нравится качество выполнения заявки   1671 non-null   float64
 3   Нравится качество работы сотрудников  1671 non-null   int64  
 4   Вопрос решен                          1671 non-null   float64
 5   Понравилось выполнение заявки         1671 non-null   float64
dtypes: float64(3), int64(2), object(1)
memory usage: 91.4+ KB


In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    df[text_col].tolist(),
    df[label_cols].values,
    test_size=0.2,
    random_state=52
)


In [12]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

class CommentDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = tokenizer(self.texts[idx],
                             truncation=True,
                             padding='max_length',
                             max_length=128,
                             return_tensors="pt")
        item = {key: val.squeeze() for key, val in encoding.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

train_dataset = CommentDataset(X_train, y_train)
test_dataset = CommentDataset(X_test, y_test)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [13]:
import os
os.environ["WANDB_DISABLED"] = "true"


model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=len(label_cols),
    problem_type="multi_label_classification"
)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=8,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs",
    logging_steps=100,
    #evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
100,0.406700
200,0.277200
300,0.249400
400,0.222700
500,0.200700
600,0.161200
700,0.158100
800,0.134100
900,0.108400
1000,0.114200


TrainOutput(global_step=1336, training_loss=0.17158704549966458, metrics={'train_runtime': 330.1741, 'train_samples_per_second': 32.371, 'train_steps_per_second': 4.046, 'total_flos': 703051676663808.0, 'train_loss': 0.17158704549966458, 'epoch': 8.0})

In [14]:
preds = trainer.predict(test_dataset)
y_pred = (preds.predictions > 0.5).astype(int)
print(classification_report(y_test, y_pred, target_names=label_cols))


                                      precision    recall  f1-score   support

  Нравится скорость отработки заявок       0.94      0.97      0.96       153
 Нравится качество выполнения заявки       0.81      0.54      0.65        24
Нравится качество работы сотрудников       0.65      0.59      0.62        51
                        Вопрос решен       0.98      0.96      0.97       257
       Понравилось выполнение заявки       0.69      0.49      0.57        49

                           micro avg       0.91      0.87      0.89       534
                           macro avg       0.82      0.71      0.75       534
                        weighted avg       0.91      0.87      0.88       534
                         samples avg       0.93      0.90      0.90       534



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



```
                precision    recall  f1-score   support

  Нравится скорость отработки заявок       0.99      0.96      0.97       162
 Нравится качество выполнения заявки       0.81      0.78      0.80        55
Нравится качество работы сотрудников       0.73      0.51      0.60        43
                        Вопрос решен       0.96      0.97      0.97       250

                           micro avg       0.94      0.91      0.92       510
                           macro avg       0.87      0.81      0.83       510
                        weighted avg       0.94      0.91      0.92       510
                         samples avg       0.94      0.92      0.92       510

```



In [15]:
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = torch.sigmoid(torch.tensor(logits)).numpy()

    # Binarize с порогом 0.5
    preds = (probs >= 0.5).astype(int)

    auc = roc_auc_score(labels, probs, average="macro")
    f1 = f1_score(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)

    return {
        "accuracy": acc,
        "f1": f1,
        "roc_auc": auc
    }
print(compute_metrics((preds.predictions, y_test)))

{'accuracy': 0.7402985074626866, 'f1': 0.762054531673346, 'roc_auc': np.float64(0.928149424724371)}


'accuracy': 0.8246153846153846, 'f1': 0.8460273006101483, 'roc_auc': np.float64(0.952922541104103)
